<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">imports</p>

In [ ]:
import os
import sys
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import polars as pl
import pandas as pd

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
import plotly.colors as pc
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import lightgbm as lgb
import kaggle_evaluation.mcts_inference_server
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error as mse

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">configuration class</p>

In [ ]:
class CFG:
    
    train_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv')
    batch_size = 65536
    
    n_features = 200
    early_stop = 100
    n_splits = 5
    color = '#C9A9A6'
    
    lgb_p = {
        'objective': 'regression',
        'num_iterations': 20000,
        'min_child_samples': 24,
        'learning_rate': 0.03,
        'extra_trees': True,
        'reg_lambda': 0.8,
        'reg_alpha': 0.1,
        'num_leaves': 64,
        'metric': 'rmse',
        'device': 'cpu',
        'max_depth': 8,
        'max_bin': 128,
        'verbose': -1,
        'seed': 42
    }

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">feature engineering class</p>

In [ ]:
class FE:
    
    def __init__(self, batch_size):
        self.batch_size = batch_size
        
    def drop_cols(self, df, bad_cols=None): # bad_cols must be provided when processing the test data
        
        cols = ['Id', 
                'LudRules', 
                'EnglishRules',
                'num_wins_agent1',
                'num_draws_agent1',
                'num_losses_agent1',
                
                # Derived from feature importance with 20 iterations training
               'PlayerValue', 'IsPieceAt', 'NumPerimeterSites', 'SetPending', 'BoardCoverageFull', 'Symbols', 'ReplacementCapture', 'PolygonShape', 'NoMovesWinFrequency', 'NumPlayPhase', 'ControlFlowStatement', 'SlideDecisionFrequency', 'MoveDistanceChangeLineBestFit', 'NumConcaveCorners', 'Dice', 'GreaterThan', 'PieceNumberChangeAverage', 'Multiplication', 'Connection', 'Directions', 'LeapDecisionToEmptyFrequency', 'BoardStyle', 'PieceNumberMaxDecrease', 'ChessStyle', 'TwoSitesMoves', 'MoveDistanceAverage', 'SlideDecisionToEmpty', 'PromotionEffect', 'BoardSitesOccupiedMaxDecrease', 'ScoringEndFrequency', 'SowCW', 'NumColumns', 'NoMovesEnd', 'SowCCW', 'FromToDecisionEnemyFrequency', 'HopDecisionMoreThanOneFrequency', 'ReachEnd', 'SowCapture', 'NumStartComponents', 'BoardSitesOccupiedChangeAverage', 'Misere', 'StackType', 'ReachWin', 'ReachEndFrequency', 'MancalaFourRows', 'NumContainers', 'DiamondShape', 'HexTiling', 'StepDecisionToEmpty', 'PassDecisionFrequency', 'NoMovesWin', 'NoMovesDraw', 'ConcentricTiling', 'NoPieceMover', 'SameLayerDirection', 'PatternEnd', 'RemoveDecisionFrequency', 'InitialScore', 'MoveDistanceChangeAverage', 'Tile', 'MoveDistanceChangeNumTimes', 'DecisionFactorChangeLineBestFit', 'AddDecision', 'CheckmateWin', 'OpeningContract', 'BranchingFactorChangeNumTimesn', 'EliminatePiecesEnd', 'PieceNumberChangeLineBestFit', 'PieceNumberMaxIncrease', 'LineWin', 'SquarePyramidalShape', 'LineEndFrequency', 'SpaceEnd', 'BoardSitesOccupiedMaxIncrease', 'BoardSitesOccupiedChangeLineBestFit', 'BackwardLeftDirection', 'MoveDistanceVariance', 'NoOwnPiecesEndFrequency', 'LineEnd', 'RookComponent', 'BallComponent', 'SurroundCapture', 'ForgetValues', 'HopDecisionFrequency', 'GoStyle', 'AlquerqueBoardWithFourTriangles', 'Variable', 'TaflComponent', 'FortyStonesWithFourGapsBoard', 'Priority', 'TaflStyle', 'MaxMovesInTurn', 'HopDecisionMoreThanOne', 'SowWithEffect', 'LeapDecisionFrequency', 'SowCaptureFrequency', 'InternalCounter', 'InterveneCapture', 'NumTopSites', 'PawnComponent', 'FromToDecisionWithinBoardFrequency', 'QueenComponent', 'CopyContext', 'ForEachPiece', 'MancalaSixRows', 'Trigger', 'Visual', 'KnightComponent', 'Pattern', 'TurnKo', 'NumOffDiagonalDirections', 'NumComponentsTypePerPlayer', 'NumDice', 'StrategoComponent', 'Repetition', 'CheckersComponent', 'DiscComponent', 'BranchingFactorChangeSign', 'JanggiComponent', 'SwapOption', 'SpiralShape', 'StarShape', 'Stack', 'Meta', 'PositionalSuperko', 'InitialCost', 'ShowPieceValue', 'BoardSitesOccupiedChangeNumTimes', 'AsymmetricForces', 'SpiralTiling', 'CircleTiling', 'SwapPlayersDecisionFrequency', 'Efficiency', 'SetInternalCounter', 'SwapPlayersDecision', 'VoteDecision', 'SemiRegularTiling', 'ThreeMensMorrisBoardWithTwoTriangles', 'BoardCoverageDefault', 'Moves', 'TriangleTiling', 'RememberValues', 'VisitedSites', 'SetSiteState', 'StackState', 'StateType', 'XiangqiComponent', 'Implementation', 'InitialRandomPlacement', 'NineMensMorrisBoard', 'AutoMove', 'NumComponentsType', 'PachisiBoard', 'ShogiComponent', 'Minimum', 'ShibumiStyle', 'Math', 'PenAndPaperStyle', 'PieceRotation', 'PieceValue', 'Addition', 'MancalaStyle', 'Division', 'Modulo', 'Absolute', 'Exponentiation', 'GraphStyle', 'Maximum', 'ScoreDifferenceMaxIncrease', 'Comparison', 'Style', 'Piece', 'KintsBoard', 'Draw', 'Algorithmics', 'Cooperation', 'NumCentreSites', 'TriangleShape', 'Team', 'Even', 'NumPhasesBoard', 'ScoreDifferenceMaxDecrease', 'ScoreDifferenceChangeLineBestFit', 'PloyComponent', 'PieceDirection', 'LargePiece', 'FairyChessComponent', 'DiceD6', 'BishopComponent', 'Odd', 'MoveDistanceChangeSign', 'KingComponent', 'MoveDistanceMaxIncrease', 'MoveDistanceMaxDecrease', 'DiceD4', 'NumLayers', 'AsymmetricPiecesType', 'NoBoard', 'BackgammonStyle', 'PieceNumberChangeNumTimes', 'SurakartaStyle', 'ScoreDifferenceAverage', 'ScoreDifferenceMedian', 'ScoreDifferenceMaximum', 'TableStyle', 'ShogiStyle', 'ScoreDifferenceVariance', 'ScoreDifferenceChangeAverage', 'ScoreDifferenceChangeSign', 'XiangqiStyle', 'JanggiStyle', 'PassDecision', 'HopDecisionFriendToFriendFrequency', 'NoProgressDrawFrequency', 'SetCountFrequency', 'CaptureSequenceFrequency', 'CaptureSequence', 'SurroundCaptureFrequency', 'InterveneCaptureFrequency', 'CustodialCaptureFrequency', 'EncloseCaptureFrequency', 'EncloseCapture', 'DirectionCaptureFrequency', 'DirectionCapture', 'HopCaptureMoreThanOne', 'LeapDecision', 'ReplacementCaptureFrequency', 'LeapDecisionToEmpty', 'MaxDistance', 'ByDieMove', 'LeapEffect', 'SlideEffect', 'StepEffect', 'SetRotationFrequency', 'MancalaCircular', 'Group', 'SlideDecisionToFriendFrequency', 'NoPieceNext', 'AllDirections', 'ProgressCheck', 'CountPiecesNextComparison', 'CountPiecesMoverComparison', 'LineOfSight', 'StepDecisionToFriendFrequency', 'Threat', 'NoTargetPiece', 'StepDecisionToEnemy', 'Loop', 'SlideDecisionToEmptyFrequency', 'NoMovesMover', 'SlideDecisionToEnemy', 'SlideDecisionToEnemyFrequency', 'SlideDecisionToFriend', 'Fill', 'Territory', 'PathExtent', 'SetRotation', 'SetCount', 'NoProgressDraw', 'SetValueFrequency', 'ProposeEffect', 'Roll', 'PassEffect', 'SwapPlayersEffect', 'VoteEffect', 'MovesEffects', 'ShootDecisionFrequency', 'ShootDecision', 'SwapPiecesDecisionFrequency', 'SwapPiecesDecision', 'FromToDecisionFriendFrequency', 'FromToDecisionFriend', 'FromToDecisionEnemy', 'HopDecisionFriendToEmpty', 'FromToDecisionFrequency', 'HopDecisionFriendToEmptyFrequency', 'HopDecisionFriendToEnemyFrequency', 'HopDecisionFriendToEnemy', 'HopDecisionEnemyToEnemyFrequency', 'ProposeEffectFrequency', 'AddEffect', 'AddEffectFrequency', 'PushEffect', 'SetValue', 'MoveAgain', 'LeapDecisionToEnemy', 'SetNextPlayer', 'LeapDecisionToEnemyFrequency', 'FlipFrequency', 'Flip', 'PushEffectFrequency', 'AlquerqueBoardWithEightTriangles', 'Sow', 'PromotionEffectFrequency', 'SowOriginFirst', 'SowSkip', 'SowProperties', 'SowBacktrackingFrequency', 'SowBacktracking', 'SowRemoveFrequency', 'SowRemove', 'RotationDecisionFrequency', 'RotationDecision', 'RotationalDirection', 'ForwardDirection', 'NoOwnPiecesWinFrequency', 'NoOwnPiecesWin', 'NoOwnPiecesEnd', 'ProposeDecision', 'EliminatePiecesDrawFrequency', 'EliminatePiecesDraw', 'EliminatePiecesLossFrequency', 'EliminatePiecesLoss', 'EliminatePiecesWinFrequency', 'EliminatePiecesWin', 'ProposeDecisionFrequency', 'NoTargetPieceWinFrequency', 'NoTargetPieceWin', 'NoTargetPieceEndFrequency', 'NoTargetPieceEnd', 'CheckmateWinFrequency', 'CheckmateFrequency', 'Checkmate', 'TerritoryWinFrequency', 'HopDecisionEnemyToEnemy', 'NoOwnPiecesLossFrequency', 'FillEnd', 'ScoringWinFrequency', 'NoProgressEndFrequency', 'NoProgressEnd', 'NoMovesDrawFrequency', 'NoMovesLossFrequency', 'NoMovesLoss', 'ScoringDraw', 'ScoringLossFrequency', 'ScoringLoss', 'ScoringWin', 'FillEndFrequency', 'ScoringEnd', 'ReachDrawFrequency', 'ReachDraw', 'ReachLossFrequency', 'ReachLoss', 'ReachWinFrequency', 'FillWinFrequency', 'FillWin', 'TerritoryWin', 'TerritoryEndFrequency', 'TerritoryEnd', 'SameDirection', 'LineDraw', 'LineLossFrequency', 'LineLoss', 'LineWinFrequency', 'Boardless', 'SumDice', 'MancalaThreeRows', 'OppositeDirection', 'BackwardRightDirection', 'ConnectionEndFrequency', 'ForwardRightDirection', 'LeftwardsDirection', 'RightwardsDirection', 'LeftwardDirection', 'PromotionDecisionFrequency', 'BackwardsDirection', 'ForwardsDirection', 'BackwardDirection', 'ConnectionEnd', 'ConnectionWin', 'PathExtentLoss', 'LoopEndFrequency', 'PathExtentWin', 'PathExtentEnd', 'PatternWinFrequency', 'PatternWin', 'PatternEndFrequency', 'LoopLoss', 'LoopWinFrequency', 'LoopWin', 'LoopEnd', 'ConnectionWinFrequency', 'GroupDraw', 'GroupLoss', 'GroupWinFrequency', 'GroupWin', 'GroupEndFrequency', 'GroupEnd', 'ConnectionLossFrequency', 'ConnectionLoss', 'NoOwnPiecesLoss']
        
        df = df.drop([col for col in cols if col in df.columns])
        df = df.drop([col for col in df.columns if df.select(pl.col(col).null_count()).item() == df.height])
        
        bad_cols = [col for col in df.columns if df.select(pl.col(col).n_unique()).item() == 1] if bad_cols is None else bad_cols
        df = df.drop(bad_cols)
        
        return df, bad_cols
    
    def cast_datatypes(self, df):
        
        cat_cols = ['GameRulesetName', 'agent1', 'agent2']
        df = df.with_columns([pl.col(col).cast(pl.String) for col in cat_cols])   
        
        for col in df.columns:
            if col not in cat_cols:
            
                val = df.select(pl.col(col).drop_nulls().first()).item()
                df = df.with_columns(pl.col(col).cast(pl.Int16) if isinstance(val, int) else pl.col(col).cast(pl.Float32))   
            
        return df     
    
    def info(self, df):
        
        print(f'Shape: {df.shape}')   
        mem = df.estimated_size() / 1024**2
        print('Memory usage: {:.2f} MB\n'.format(mem))
        
    def apply_fe(self, path):
        
        df = pl.read_csv(path, batch_size=self.batch_size)
        
        df, bad_cols = self.drop_cols(df)
        df = self.cast_datatypes(df)
        self.info(df)
        
        cat_cols = [col for col in df.columns if df[col].dtype == pl.String]
        
        return df, bad_cols, cat_cols

In [ ]:
fe = FE(CFG.batch_size)

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">exploratory data analysis</p>

In [ ]:
class EDA:
    
    def __init__(self, df, color):
        self.df = df  
        self.color = color  

    def template(self, fig, title):
        
        fig.update_layout(
            title=title,
            title_x=0.5, 
            plot_bgcolor='rgba(0,0,0,0)', 
            paper_bgcolor='rgba(0,0,0,0)',  
            font=dict(color='#7f7f7f'),
            margin=dict(l=90, r=90, t=90, b=90), 
            height=900  
        )
        
        return fig
    
    def target_distribution(self):
        
        target_distribution = self.df['utility_agent1'].value_counts().sort_index()

        fig = px.histogram(
            self.df,
            x='utility_agent1',
            nbins=50, 
            title='Distribution of Agent 1 Utility',  
            color_discrete_sequence=[self.color]  
        )

        fig.update_layout(
            xaxis_title='Utility of Agent 1',
            yaxis_title='Count', 
            bargap=0.1  
        )

        fig.update_traces(hovertemplate='Utility: %{x:.3f}<br>Count: %{y:,}')
        fig = self.template(fig, 'Distribution of Agent 1 Utility')
        fig.show()
    
    def value_distribution(self):

        binary_cols = [] 
        other_cols = [] 

        for col in self.df.columns:
            if self.df[col].nunique() == 2:
                binary_cols.append(col)
            elif self.df[col].nunique() > 2:
                other_cols.append(col)

        labels = ['2 values', '>2 values']
        values = [len(binary_cols), len(other_cols)]
        percentages = [round(count / len(self.df.columns) * 100) for count in values]

        hover_text = [f'Case: {label}<br>Count: {count}<br>Percent: {percentage}%' 
                      for label, count, percentage in zip(labels, values, percentages)]

        fig = px.pie(
            values=values,
            names=labels,
            title='Distribution of Column Types',
            color_discrete_sequence=px.colors.sequential.Redor,
            custom_data=[hover_text]
        )
        
        fig.update_traces(hovertemplate='%{customdata[0]}<extra></extra>')
        fig = self.template(fig, 'Distribution of Column Types')
        fig.show()

In [ ]:
train_data, _, _ = fe.apply_fe(CFG.train_path)
train_data = train_data.to_pandas()
display(train_data.head())

In [ ]:
eda = EDA(train_data, CFG.color)

In [ ]:
eda.target_distribution()

In [ ]:
eda.value_distribution()

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">model development class</p>

In [ ]:
class MD:
    
    def __init__(self, n_features, early_stop, n_splits, lgb_p, color):
        self.n_features = n_features
        self.early_stop = early_stop
        self.n_splits = n_splits
        self.lgb_p = lgb_p
        self.color = color
        
    def plot_cv(self, fold_scores, title):
        
        fold_scores = [round(score, 3) for score in fold_scores]
        mean_score = round(np.mean(fold_scores), 3)
        std_score = round(np.std(fold_scores), 3)

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x = list(range(1, len(fold_scores) + 1)),
            y = fold_scores,
            mode = 'markers', 
            name = 'Fold Scores',
            marker = dict(size = 24, color=self.color, symbol='diamond'),
            text = [f'{score:.3f}' for score in fold_scores],
            hovertemplate = 'Fold %{x}: %{text}<extra></extra>',
            hoverlabel=dict(font=dict(size=16))  
        ))

        fig.add_trace(go.Scatter(
            x = [1, len(fold_scores)],
            y = [mean_score, mean_score],
            mode = 'lines',
            name = f'Mean: {mean_score:.3f}',
            line = dict(dash = 'dash', color = '#FFBF00'),
            hoverinfo = 'none'
        ))

        fig.update_layout(
            title = f'{title} | Cross-Validation RMSE Scores | Variation of CV scores: {mean_score} ± {std_score}',
            xaxis_title = 'Fold',
            yaxis_title = 'RMSE Score',
            plot_bgcolor = 'rgba(0,0,0,0)',
            paper_bgcolor = 'rgba(0,0,0,0)',
            xaxis = dict(
                gridcolor = 'lightgray',
                tickmode = 'linear',
                tick0 = 1,
                dtick = 1,
                range = [0.5, len(fold_scores) + 0.5]
            ),
            yaxis = dict(gridcolor = 'lightgray')
        )

        fig.show() 
        
    def train_lgb(self, data, cat_cols, title):
        
        X = data.drop(['utility_agent1'], axis=1)
        y = data['utility_agent1']
        group = data['GameRulesetName']
        
        for col in cat_cols:
            X[col] = X[col].astype('category')
        
        cv = GroupKFold(self.n_splits)
        
        models, scores = [], []
        oof_preds = np.zeros(len(X))
        
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y, group)):
            
            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
            model = lgb.LGBMRegressor(**self.lgb_p)
            model.fit(X_train, y_train,
                      eval_set=[(X_valid, y_valid)],
                      eval_metric='rmse',
                      callbacks=[lgb.early_stopping(self.early_stop, verbose=0), 
                                 lgb.log_evaluation(0)])
    
            models.append(model)
            
            oof_preds[valid_index] = model.predict(X_valid)
            score = mse(y_valid, oof_preds[valid_index], squared=False)
            scores.append(score)
        
        self.plot_cv(scores, title)
        
        return models

    def infer_lgb(self, data, cat_cols, models):

        for col in cat_cols:
            data[col] = data[col].astype('category')

        return np.mean([model.predict(data) for model in models], axis=0)
    
    def feature_importance(self, data, cat_cols, title):
        
        models = self.train_lgb(data, cat_cols, title)
        
        feature_importances = np.zeros(len(data.columns) - 1)
        for model in models:
            feature_importances += model.feature_importances_ / len(models)
        
        feature_importance = pd.DataFrame({
            'feature': [col for col in data.columns if col != 'utility_agent1'],
            'importance': feature_importances
        })
        
        feature_importance = feature_importance.sort_values('importance', ascending=False).reset_index(drop=True)
        display(feature_importance)
        
        drop_features = feature_importance.loc[self.n_features:, 'feature'].tolist()
        
        return drop_features

In [ ]:
md = MD(CFG.n_features, CFG.early_stop, CFG.n_splits, CFG.lgb_p, CFG.color)

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">feature importance</p>

In [ ]:
#train, _, cat_cols = fe.apply_fe(CFG.train_path)
#train = train.to_pandas()

In [ ]:
#drop_features = md.feature_importance(train, cat_cols, 'LightGBM')

In [ ]:
#del train, cat_cols

In [ ]:
#print(drop_features)

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">trainer function</p>

In [ ]:
def train_model():
    
    global bad_cols, cat_cols, lgb_models
    
    train, bad_cols, cat_cols = fe.apply_fe(CFG.train_path)
    train = train.to_pandas()
    lgb_models = md.train_lgb(train, cat_cols, 'LightGBM')

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">inference function</p>

In [ ]:
counter = 0
def predict(test, submission):
    
    global counter
    
    if counter == 0:
        train_model() 
        
    counter += 1
    
    test, _ = fe.drop_cols(test, bad_cols)
    test = fe.cast_datatypes(test)
    test = test.to_pandas()
    
    return submission.with_columns(pl.Series('utility_agent1', md.infer_lgb(test, cat_cols, lgb_models)))

<p style="background-color: #1B1212; font-size: 300%; text-align: center; border-radius: 40px 40px; color: #C9A9A6; font-weight: bold; font-family: 'Cinzel', serif; text-transform: uppercase; border: 4px solid #C9A9A6;">call the gateway server</p>

In [ ]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )